In [1]:
import pandas as pd
import numpy as np

In [2]:
def calculate_symbol_metrics(meta_sp500, meta_nasdaq, meta_commodities, meta_stoxx600, prices):
    # Combine all metadata datasets into one DataFrame with a single 'Symbol' column
    all_meta = pd.concat([
        meta_sp500[['Symbol']],
        meta_nasdaq[['Symbol']],
        meta_commodities[['Symbol']],
        meta_stoxx600[['Symbol']]
    ])
    
    # Ensure no duplicate symbols and reset the index
    all_meta = all_meta.drop_duplicates().reset_index(drop=True)

    # Create a DataFrame to store the calculated metrics
    metrics = pd.DataFrame(index=all_meta['Symbol'], columns=['Average Return', 'Risk', 'Return-to-Risk'])

    # Iterate through each symbol and calculate metrics
    for symbol in all_meta['Symbol']:
        if symbol in prices.columns:
            # Get the price series for the symbol
            price_series = prices[symbol].dropna()
            
            # Calculate monthly returns
            returns = price_series.pct_change().dropna()
            
            # Calculate average return and risk (std deviation)
            avg_return = returns.mean()
            risk = returns.std()
            
            # Store the metrics
            metrics.loc[symbol, 'Average Return'] = avg_return
            metrics.loc[symbol, 'Risk'] = risk
            metrics.loc[symbol, 'Return-to-Risk'] = avg_return / risk if risk != 0 else 0

    # Drop rows with NaN values (symbols with no price data)
    metrics = metrics.dropna()

    # Convert metrics columns to numeric
    metrics = metrics.apply(pd.to_numeric)

    # Rank the symbols based on Return-to-Risk ratio (descending order)
    metrics = metrics.sort_values(by='Return-to-Risk', ascending=False)

    return metrics

In [3]:
meta_sp500 = pd.read_csv("../data/tickers_sp500.csv")
meta_nasdaq = pd.read_csv("../data/tickers_nasdaq.csv")
meta_commodities = pd.read_csv("../data/tickers_commodities.csv")
meta_stoxx600 = pd.read_csv("../data/tickers_stoxx600.csv")
prices = pd.read_csv("../data/monthly_prices.csv", index_col=0, parse_dates=True)

In [4]:
metrics = calculate_symbol_metrics(meta_sp500, meta_nasdaq, meta_commodities, meta_stoxx600, prices)

In [5]:
metrics.head(20)

,Average Return,Risk,Return-to-Risk
Symbol,,,
BHRB,0.137384,0.272141,0.504825
LLY,0.032817,0.079903,0.410709
NVO,0.022720,0.060802,0.373671
DECK,0.032019,0.088519,0.361718
AJG,0.020842,0.059970,0.347547
MSFT,0.021051,0.061439,0.342631
TRI,0.017748,0.052921,0.335370
ENPH,0.077903,0.234543,0.332147
CDNS,0.027550,0.083660,0.329304
